# 04 Virtual point transformation (VPT)
The classic interface reduction approach based on projecting the measurements on a space of rigid interface deformation modes is embedded in `pyFBS`. For more details on the virtual point transformation, refer to the documentation.

In [1]:
import pyFBS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Numerical example
First load a predefined datasets from an example directory. 

In [2]:
pyFBS.download_lab_testbench()

100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 8031.22it/s]

In [3]:
stl = r"./lab_testbench/STL/B.stl"
xlsx_pos = r"./lab_testbench/Measurements/ammeasurements.xlsx"

full_file = r"./lab_testbench/FEM/B.full"
ress_file = r"./lab_testbench/FEM/B.rst"

#### MK model
Prepare a MK model for the FRF synthetization. For more details check-out the *03_FRF_synthetization.ipynb* example. 

In [ ]:
MK = pyFBS.MK_model(ress_file,full_file,no_modes = 100,recalculate = False, scale=1000)

[100]
[160]
[3]


Add a STL and a coresponding mesh from FEM to the 3D view:

In [5]:
view3D = pyFBS.view3D(show_origin= True)
mesh = view3D.add_stl(stl,name = "engine_mount",color = "#8FB1CC",opacity = .1)
view3D.plot.add_mesh(MK.mesh, scalars = np.zeros(MK.mesh.points.shape[0]),show_scalar_bar = False,name = "mesh",cmap = "coolwarm",show_edges = True);

To evaluate the FEM model the mode shapes can be animated: 

In [6]:
select_mode = 6
_modeshape = MK.get_modeshape(select_mode)

mode_dict = pyFBS.utility.dict_animation(_modeshape,"modeshape",pts = MK.pts.copy(),mesh = MK.mesh)
view3D.add_modeshape(mode_dict,run_animation = True)

The mode shape deformation can also be cleared from the display:

In [7]:
view3D.clear_modeshape()

Load the channel and datasets. Following the theoretical explanation, 3 tri-axial sensors (with 9 channels in total) and 9 unidirectional excitations are positioned on a structure. Positioning can be performed using interactive positioning or imported from Excel datasheets (as in this case):

In [8]:
df_imp = pd.read_excel(xlsx_pos, sheet_name='Impacts_B')
df_chn = pd.read_excel(xlsx_pos, sheet_name='Channels_B')

#view3D.show_imp(df_imp,overwrite = True)
#view3D.show_chn(df_chn)

Currently only "snap-to-node" FRF synthetization is supported. Therefore, the locations of the channels and impacts have to updated to snap to the nearest node from FEM model.

In [9]:
df_chn_up = MK.update_locations_df(df_chn)
df_imp_up = MK.update_locations_df(df_imp)

#view3D.show_chn(df_chn_up, color = "k",overwrite = False)
#view3D.show_imp(df_imp_up, color = "k",overwrite = False)

<div class="alert alert-block alert-warning">
    <i class="fa fa-exclamation-triangle" aria-hidden="true"></i>
    <b>Warning!</b> When performing numerical cases always make sure channels and impacts are aligned with FE nodes!
</div>

#### FRF synthetization
Finally, the FRFs synthetization at the channel/impact DoFs can be performed:

In [10]:
MK.FRF_synth(df_chn_up,df_imp_up,f_start = 0,modal_damping = 0.003,frf_type = "accelerance")

## Virtual point transformation 
For the VPT the information on the position/orientation of the virtual point is required (``df_vp`` and ``df_vpref`` for output and input respectively), together with the corresponding channel/impact information (``df_chn_up`` and ``df_imp_up`` for output and input respectively). Again, this can be done with interactive positioning or by importing Excel dataframe:

In [11]:
df_vp = pd.read_excel(xlsx_pos, sheet_name='VP_Channels')
df_vpref = pd.read_excel(xlsx_pos, sheet_name='VP_RefChannels')

The `Description` tells which DoFs you would like to reconstruct using VPT. `Grouping` number organizes which impacts and sensors belong to which virtual point. Arbitrary number of VPs can be reconstructed at once if grouping numbers are properly defined.

In [12]:
df_vp

Grouping                 Quantity     Unit    Name Description  Position_1  \
0         1             Acceleration    m/s^2  VP1_ux          ux   38.895369   
1         1             Acceleration    m/s^2  VP1_uy          uy   38.895369   
2         1             Acceleration    m/s^2  VP1_uz          uz   38.895369   
3         1  Rotational acceleration  rad/s^2  VP1_tx          tx   38.895369   
4         1  Rotational acceleration  rad/s^2  VP1_ty          ty   38.895369   
5         1  Rotational acceleration  rad/s^2  VP1_tz          tz   38.895369   

   Position_2  Position_3  Direction_1  Direction_2  Direction_3  
0   348.10701         7.0            1            0            0  
1   348.10701         7.0            0            1            0  
2   348.10701         7.0            0            0            1  
3   348.10701         7.0            1            0            0  
4   348.10701         7.0            0            1            0  
5   348.10701         7.0            0            0            1

Once channels, impacts, and virtual points are correctly positioned, VPT model can be created:

In [13]:
vpt = pyFBS.VPT(df_chn_up,df_imp_up,df_vp,df_vpref)

Model ``vpt`` contains transformation matrices $\mathbf{T}_{\mathrm{u}}$ and $\mathbf{T}_{\mathrm{f}}$ based on relative sensors and impacts positions and orientations with regards to VP. Transformation matrices can be accessed in the following way:

In [14]:
Tu = vpt.Tu
Tf = vpt.Tf

Full IDM matrix can also be accessed with `pyFBS`:

In [15]:
Ru = vpt.Ru

The application of the VPT to the FRFs matrix ``FRF`` is applied :

In [16]:
vpt.apply_VPT(freq = MK.freq, FRF = MK.FRF)

Virtual point FRFs can be neatly displayed using wrapper function directly from ``pyFBS``:

In [17]:
pyFBS.plot_FRF(vpt.freq, vpt.vptData[:,:6,:6])

alt.HConcatChart(...)

#### Consistency of the VPT
The evaluation of consistency of the VPT is based on the grouping number:

In [18]:
vpt.consistency([1],[1])

Visualisation of the specific channel/sensor and impact consistency can be done in a following way:

In [19]:
spec_chn = pyFBS.barchart(np.arange(1,10,1), vpt.specific_sensor, color='steelblue', title='Specific Channel Consistency')
spec_imp = pyFBS.barchart(np.arange(1,10,1), vpt.specific_impact, color='firebrick', title='Specific Impact Consistency')
spec_chn | spec_imp

alt.HConcatChart(...)

Wrapper functions can also be used to display overall channel/sensor and impact consistency:

In [20]:
pyFBS.plot_coh(vpt.freq, vpt.overall_sensor, color='steelblue', title='Overall Channel Consistency')

alt.Chart(...)

In [21]:
pyFBS.plot_coh(vpt.freq, vpt.overall_impact, color='firebrick', title='Overall Impact Consistency')

alt.Chart(...)

Reciprocity check is also already implemented in `pyFBS`:

In [22]:
coh_crit = pyFBS.utility.coh_on_FRF(vpt.vptData[:,:6,:6])

In [24]:
pyFBS.imshow(coh_crit)

alt.Chart(...)

<div class="alert alert-block alert-success">
    <i class="fa fa-thumbs-up" aria-hidden="true"></i>
    <b>Success!</b> With the knowledge of numerical VPT example, implementation on an experimental model is pretty much straightforward. FRF matrix, build with <code>pyFBS.MCK</code> for the numerical case, is simply replaced by actual measured FRFs.
</div>